In [3]:
import pandas as pd
import random
from datetime import datetime, timedelta

# Load datasets
mentors = pd.read_csv("Mentor.csv")  # Columns: Name, Sector 1, Sector 2, Sector 3, Index
startups = pd.read_csv("Startups.csv")  # Columns: Name, Sector, Index

# Initialize time slots
start_time = datetime.strptime("11:00 AM", "%I:%M %p")
end_time = datetime.strptime("2:00 PM", "%I:%M %p")
time_slots = []
slot_duration = timedelta(minutes=15)
gap_duration = timedelta(minutes=5)

while start_time + slot_duration <= end_time:
    time_slots.append(start_time.strftime("%I:%M %p"))
    start_time += slot_duration + gap_duration

# Helper function to assign startups iteratively
def schedule_mentoring_sessions(mentors, startups, time_slots):
    # Prepare mentor preferences
    mentor_preferences = {
        mentor["Name"]: [mentor["Sector 1"], mentor["Sector 2"], mentor["Sector 3"]]
        for _, mentor in mentors.iterrows()
    }

    # Initialize mentor schedules and startup session counts
    mentor_schedule = {mentor["Name"]: [] for _, mentor in mentors.iterrows()}
    startup_counts = {startup["Name"]: 0 for _, startup in startups.iterrows()}

    # Assign startups iteratively to mentors
    for time_slot in time_slots:
        for _, mentor in mentors.iterrows():
            mentor_name = mentor["Name"]
            preferences = mentor_preferences[mentor_name]
            weights = [40, 30, 30]

            # Filter startups based on preferences and repetition constraints
            available_startups = startups[
                (startups["Name"].map(startup_counts) < 4) &
                (startups["Sector"].isin(preferences))
            ]

            # If no startups match preferences, fallback to any available startup
            if available_startups.empty:
                available_startups = startups[startups["Name"].map(startup_counts) < 4]

            if available_startups.empty:
                continue  # Skip if no startups are available

            # Select a startup based on weighted preferences or randomly
            selected_startup = None
            if available_startups["Sector"].isin(preferences).any():
                chosen_sector = random.choices(preferences, weights=weights, k=1)[0]
                sector_startups = available_startups[available_startups["Sector"] == chosen_sector]
                if not sector_startups.empty:
                    selected_startup = sector_startups.sample(1).iloc[0]
            if selected_startup is None:
                selected_startup = available_startups.sample(1).iloc[0]

            startup_name = selected_startup["Name"]
            startup_sector = selected_startup["Sector"]

            # Assign the startup to the mentor for the current time slot
            mentor_schedule[mentor_name].append({
                "Name": startup_name,
                "Sector": startup_sector,
                "Time Slot": time_slot
            })
            startup_counts[startup_name] += 1

    return mentor_schedule, startup_counts

# Schedule mentoring sessions
schedule, startup_counts = schedule_mentoring_sessions(mentors, startups, time_slots)

# Display the schedule
print("Mentoring Schedule:")
for _, mentor in mentors.iterrows():
    print(f"\nMentor: {mentor['Name']}")
    print(f"Preferences: {mentor['Sector 1']}, {mentor['Sector 2']}, {mentor['Sector 3']}")
    print("Scheduled Startups:")
    for session in schedule[mentor["Name"]]:
        print(f"  - {session['Name']} ({session['Sector']}) at {session['Time Slot']}")

# Display startup counts
print("\nStartup Session Counts:")
for startup, count in startup_counts.items():
    print(f"{startup}: {count} sessions")


Mentoring Schedule:

Mentor: Mentor 1
Preferences: B2B Software, Marketplace, Media/Media Tech
Scheduled Startups:
  - Startup 10 (B2B Software) at 11:00 AM
  - Startup 60 (B2B Software) at 11:20 AM
  - Startup 121 (Media/Media Tech) at 11:40 AM
  - Startup 69 (Media/Media Tech) at 12:00 PM
  - Startup 31 (Marketplace) at 12:20 PM
  - Startup 139 (Marketplace) at 12:40 PM
  - Startup 91 (Media/Media Tech) at 01:00 PM
  - Startup 81 (Media/Media Tech) at 01:20 PM
  - Startup 141 (Marketplace) at 01:40 PM

Mentor: Mentor 2
Preferences: Consumer goods and services, HR Tec, Government
Scheduled Startups:
  - Startup 86 (Government) at 11:00 AM
  - Startup 6 (Consumer goods and services) at 11:20 AM
  - Startup 49 (HR Tec) at 11:40 AM
  - Startup 49 (HR Tec) at 12:00 PM
  - Startup 143 (Government) at 12:20 PM
  - Startup 129 (HR Tec) at 12:40 PM
  - Startup 82 (Consumer goods and services) at 01:00 PM
  - Startup 129 (HR Tec) at 01:20 PM
  - Startup 14 (Consumer goods and services) at 01:4